In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [33]:
dados=pd.read_csv("DadosDeCancer.csv")

In [34]:
dados2=pd.DataFrame(np.copy(dados),columns=dados.columns)

In [35]:
dados2.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,no


# A única variável que é numérica está com o seu tipo alterado pois veio de um array numpy, irei consertar em seguida.

In [36]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null object
breast        286 non-null object
irradiat      286 non-null object
dtypes: object(9)
memory usage: 20.2+ KB


In [37]:
#Ajeitando o tipo da variável deg-malig
dados2["deg-malig"]=np.array([int(i) for i in dados2["deg-malig"]])

In [38]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null int32
breast        286 non-null object
irradiat      286 non-null object
dtypes: int32(1), object(8)
memory usage: 19.1+ KB


# Mudando as classes para 0 e 1.

In [39]:
dados2["Classe"]=np.array([1 if i=="no-recurrence-events" else 0 for i in dados2["class"]])

In [40]:
dados2["Classe"]

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
256    0
257    0
258    0
259    0
260    0
261    0
262    0
263    0
264    0
265    0
266    0
267    0
268    0
269    0
270    0
271    0
272    0
273    0
274    0
275    0
276    0
277    0
278    0
279    0
280    0
281    0
282    0
283    0
284    0
285    0
Name: Classe, Length: 286, dtype: int32

# Usando a transformação OneHotEncoder para mudar as variáveis categóricas em numéricas, se baseia que em cada tipo único de cada variável fazer uma coluna de zeros e uns, 1 se o tipo único acontecer ou 0 caso contrário.

In [41]:
transformacao=OneHotEncoder(sparse=False)

# Organizando os dados necessários para o Perceptron, em que a primeira coluna é o peso do bias.

In [42]:
dados2=np.column_stack((np.full(dados2.shape[0],-1),transformacao
.fit_transform(dados2.loc[:,["age","menopause","tumor-size","inv-nodes"
,"node-caps","breast","irradiat"]]),dados2["deg-malig"],dados2["Classe"]))

In [43]:
dados2

array([[-1.,  0.,  1., ...,  0.,  3.,  1.],
       [-1.,  0.,  0., ...,  0.,  2.,  1.],
       [-1.,  0.,  0., ...,  0.,  2.,  1.],
       ...,
       [-1.,  0.,  0., ...,  0.,  1.,  0.],
       [-1.,  0.,  0., ...,  0.,  3.,  0.],
       [-1.,  0.,  0., ...,  0.,  3.,  0.]])

# Algoritmo MultiLayerPerceptron

In [44]:
class MultiLayerPerceptron:
    #O número de camadas e a quantidade de neurônios por camada é parâmetrizado na seguinte forma,
    #neuronios_por_camada é um array numpy com as quantidades de neurônios em cada camada, consequentemente
    #o tamanho desse vetor é a quantidade de camadas, e necessariamente precisa ter uma única camada de saída.
    def __init__(self,neuronios_por_camada):
        self.neuronios_por_camada=neuronios_por_camada
    
    def fit(self,xtreino,ytreino):

        quantidade_de_pesos=np.concatenate((xtreino.shape[1],self.neuronios_por_camada)
        ,axis=None) 
        #booleano criado até que a acurácia do conjunto de treinamento ultrapasse 73%
        flag=True
        while flag==True:
            # Aqui seto aleatoriamente todos os pesos das passagens de todas as camadas.
            self.pesos=np.array([np.random.normal(size=(i,j)) for i,j in np.column_stack((
                    quantidade_de_pesos[:-1],quantidade_de_pesos[1:]))])
            #Aqui eu adiciono o bias nas camadas ocultas.
            self.pesos=np.array([np.row_stack((self.pesos[i],np.repeat(
                    -np.random.normal(size=1),repeats=self.pesos[i].shape[1])))
                if i!=0 else self.pesos[i] for i in range(self.pesos.shape[0])])
            
            previsoes=[]
            #Percorrendo os indices de treinamento.
            for j in range(xtreino.shape[0]):
                #Indices das camadas ocultas e da saída.
                for i in range(self.neuronios_por_camada.shape[0]):
                    #Na camida de entrada fiz o produto matricial e depois apliquei a função sigmoide 
                    #que irá retornar a entrada da camada seguinte.
                    if i==0:
                        entrada=MultiLayerPerceptron.Sigmoide(np.concatenate((np.array([xtreino[j,:].dot(self.pesos[i])]),1),axis=None))
                    #Fazendo todas as entradas das camadas ocultas até a saída   
                    elif i!=0 and i!=self.neuronios_por_camada.shape[0]-1:
                        entrada=MultiLayerPerceptron.Sigmoide(np.concatenate((entrada.dot(self.pesos[i]),1),axis=None))
                    #Camada de saída resultando 1 se o somatório das saídas multiplicada pelos pesos
                    #for maior que zero, e 0 caso contrário.
                    else:
                        if entrada.dot(self.pesos[i])>0:
                            previsoes.append(1)
                        else:
                            previsoes.append(0)
            
            if MultiLayerPerceptron(self.neuronios_por_camada).acuracia(np.array(
                    previsoes),ytreino)>=0.72:
                flag=False
            else:
                continue
    def predict(self,xteste):
        previsoes=[]
        for j in range(xteste.shape[0]):
            for i in range(self.neuronios_por_camada.shape[0]):
                if i==0:
                    entrada=MultiLayerPerceptron.Sigmoide(np.concatenate((np.array([xteste[j,:].dot(self.pesos[i])]),1),axis=None))
                
                elif i!=0 and i!=self.neuronios_por_camada.shape[0]-1:
                    entrada=MultiLayerPerceptron.Sigmoide(np.concatenate((entrada.dot(self.pesos[i]),1),axis=None))
                else:
                    if entrada.dot(self.pesos[i])>0:
                        previsoes.append(1)
                    else:
                        previsoes.append(0)
        return(np.array(previsoes))
        
    def acuracia(self,a,b):
        cont=0
        for i in range(len(b)):
            if a[i]==b[i]:
                cont+=1
        return(np.array([np.round(cont/len(b),3)])[0])
    def matriz_de_confusao(self,real,predito):
        print(pd.crosstab(real,predito,rownames=["Classe real"]
,colnames=["Classe predita"],margins=True))
    def ReLU(w):
        return w if all(i>=0 for i in w) else 0
    def Sigmoide(u):
        return(1/(1+np.exp(-u)))

In [45]:
x=dados2[:,:-1]
y=dados2[:,-1]

# Separação do conjunto de  treino e teste estratificada.

In [46]:
xtreino,xteste,ytreino,yteste=train_test_split(x,y,test_size=0.3,stratify=y)        

# Exemplos de modelos

In [52]:
#2 camadas ocultas, cada uma com 2 neurônios.
modelo=MultiLayerPerceptron(np.array([2,2,1]))        

In [64]:
modelo.fit(xtreino,ytreino)        

In [65]:
modelo.predict(xteste)

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [66]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.721

In [67]:
modelo.matriz_de_confusao(yteste,modelo.predict(xteste))

Classe predita  0   1  All
Classe real               
0.0             4  22   26
1.0             2  58   60
All             6  80   86


In [68]:
#Uma camada oculta com 2 neurônios.
modelo=MultiLayerPerceptron(np.array([2,1]))        

In [69]:
modelo.fit(xtreino,ytreino)        

In [70]:
modelo.predict(xteste)

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [71]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.744

In [72]:
modelo.matriz_de_confusao(yteste,modelo.predict(xteste))              

Classe predita  0   1  All
Classe real               
0.0             5  21   26
1.0             1  59   60
All             6  80   86


In [73]:
#3 camadas ocultas, a primeira com 3 neurônios, a segunda com 4 e a terceira com duas.
modelo=MultiLayerPerceptron(np.array([3,4,2,1]))

In [87]:
modelo.fit(xtreino,ytreino)

In [88]:
modelo.predict(xteste)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [89]:
modelo.acuracia(yteste,modelo.predict(xteste))

0.721

In [90]:
modelo.matriz_de_confusao(yteste,modelo.predict(xteste))

Classe predita  0   1  All
Classe real               
0.0             5  21   26
1.0             3  57   60
All             8  78   86
